# Proyecto mejorara un modelo de machine learnig con scikit-learn

In [40]:
# Librerias
import numpy as np 
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest,SelectFromModel, chi2, f_classif

In [41]:
# Cargar el dataset de diabetes
diabetes = datasets.load_diabetes()

In [51]:
# Convertir a DataFrame para facilitar el analsis exploratorio
diabetes_df = pd.DataFrame(
    data=np.c_[diabetes["data"], diabetes["target"]],
    columns=diabetes["feature_names"] + ["target"],
)

In [75]:
# Convertir 'target' en categorias para clasificacion
diabetes_df['target'] = (diabetes_df['target'] > diabetes_df['target'].median()).astype(int)

In [76]:
# Division de datos en conjuntos de entrenamiento y prueba
X = diabetes_df.drop('target', axis=1)
y = diabetes_df['target']

In [121]:
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.3, random_state=42)

In [122]:
selector1 = StandardScaler()
selector1.fit(X_entrena)

StandardScaler()

In [123]:
X_entrena_escalado = selector1.transform(X_entrena)
X_prueba_escalada = selector1.transform(X_prueba)

In [124]:
X_entrena

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
225,0.030811,0.050680,0.032595,0.049415,-0.040096,-0.043589,-0.069172,0.034309,0.063015,0.003064
412,0.074401,-0.044642,0.085408,0.063187,0.014942,0.013091,0.015505,-0.002592,0.006207,0.085907
118,-0.056370,0.050680,-0.010517,0.025315,0.023198,0.040022,-0.039719,0.034309,0.020609,0.056912
114,0.023546,-0.044642,0.110198,0.063187,0.013567,-0.032942,-0.024993,0.020655,0.099241,0.023775
364,0.001751,0.050680,-0.006206,-0.019442,-0.009825,0.004949,-0.039719,0.034309,0.014821,0.098333
...,...,...,...,...,...,...,...,...,...,...
106,-0.096328,-0.044642,-0.076264,-0.043542,-0.045599,-0.034821,0.008142,-0.039493,-0.059471,-0.083920
270,0.005383,0.050680,0.030440,0.083844,-0.037344,-0.047347,0.015505,-0.039493,0.008641,0.015491
348,0.030811,-0.044642,-0.020218,-0.005670,-0.004321,-0.029497,0.078093,-0.039493,-0.010903,-0.001078
435,-0.012780,-0.044642,-0.023451,-0.040099,-0.016704,0.004636,-0.017629,-0.002592,-0.038460,-0.038357


In [125]:
selector = SelectKBest(score_func=f_classif, k=5)
X_entrena_seleccionada = selector.fit_transform(X_entrena_escalado, y_entrena)
X_prueba_seleccionada = selector.transform(X_prueba_escalada)

In [162]:
# Crear y entrenar el modelo RandomForestClassifier
modelo = RandomForestClassifier(n_estimators=100, max_features=9, max_depth=3, criterion='entropy')
modelo.fit(X_entrena_seleccionada, y_entrena)

RandomForestClassifier(criterion='entropy', max_depth=3, max_features=9)

In [163]:
# Realizar predicciones con el conjunto de pruebas
predicciones = modelo.predict(X_prueba_seleccionada)
predicciones

array([0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1])

In [170]:
puntaje = cross_val_score(modelo, X, y, cv=5)
print('Exactitud de cada particion: ', puntaje)
print('Media de exactitud: ', puntaje.mean())

Exactitud de cada particion:  [0.70786517 0.79775281 0.69318182 0.68181818 0.71590909]
Media de exactitud:  0.7193054136874361


In [173]:
# Evaluacion del modelo
puntaje = modelo.score(X_prueba_seleccionada, y_prueba)
print(f'\nPresicion del modelo: {puntaje:.2f}')


Presicion del modelo: 0.76


In [174]:
pipeline = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, max_features=9, max_depth=3, criterion='entropy'))

pipeline.fit(X_entrena_seleccionada, y_entrena)

# Realizar predicciones con el conjunto de pruebas
predicciones = pipeline.predict(X_prueba_seleccionada)
# # Evaluacion del modelo
puntaje = pipeline.score(X_prueba_seleccionada, y_prueba)
print(f"\nPresicion del modelo: {puntaje:.2f}")


Presicion del modelo: 0.79


In [148]:
# Crear y entrenar el modelo RandomForestClassifier
modelo = RandomForestClassifier()

# Evaluar parametros para mejorar el modelo
parametros = {
    "n_estimators": [50, 100, 200],
    "max_features": [5, 7, 9],
    "max_depth": [1, 2, 3, 4, 5],
    "criterion": ["gini", "entropy"],
}

grid_search = GridSearchCV(estimator=modelo, param_grid=parametros, cv=5, scoring='accuracy')

grid_search.fit(X_entrena_seleccionada, y_entrena)

print('Mejores parametros: ',grid_search.best_params_)
print('Mejor exactitud: ', grid_search.best_score_)

Mejores parametros:  {'criterion': 'entropy', 'max_depth': 3, 'max_features': 9, 'n_estimators': 100}
Mejor exactitud:  0.7410364886303542


In [149]:
selector2 = SelectFromModel(modelo)
X_nuevo = selector2.fit_transform(X_entrena, y_entrena)

- Procesamiento de datos con Scaler 
- Seleccion de mejores categorias
- Pipelines
- Evaluacion del modelo

In [96]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline

# Cargar el dataset de diabetes\n",
diabetes = datasets.load_diabetes()
# Convertir a DataFrame para un análisis exploratorio más fácil\n",
diabetes_df = pd.DataFrame(
    data=np.c_[diabetes["data"], diabetes["target"]],
    columns=diabetes["feature_names"] + ["target"],
)
# Convertir la variable 'target' en categórica para clasificación\n",
diabetes_df["target"] = (diabetes_df["target"] > diabetes_df["target"].median()).astype(
    int
)
# División de datos en conjuntos de entrenamiento y prueba\n",
X = diabetes_df.drop("target", axis=1)
y = diabetes_df["target"]
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Selección de las mejores 4 características\n",
selector = SelectKBest(score_func=f_classif, k=5)
X_entrena_seleccionada = selector.fit_transform(X_entrena, y_entrena)
X_prueba_seleccionada = selector.transform(X_prueba)

# Creación del pipeline sin la selección de características\n",
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("classifier", RandomForestClassifier(n_estimators=100, random_state=42)),
    ]
)

# Entrenamiento del modelo usando el pipeline\n",
pipeline.fit(X_entrena_seleccionada, y_entrena)

# Realización de predicciones con el conjunto de prueba\n",
predicciones = pipeline.predict(X_prueba_seleccionada)

# Evaluación del modelo\n",
puntaje = pipeline.score(X_prueba_seleccionada, y_prueba)
print(f"Precisión del modelo: {puntaje:.2f}")
# Evaluación adicional con validación cruzada\n",
puntajes_validacion_cruzada = cross_val_score(pipeline, selector.transform(X), y, cv=5)
print(f"Puntuaciones de validación cruzada: {puntajes_validacion_cruzada}")
print(
    f"Promedio de puntuación de validación cruzada: {np.mean(puntajes_validacion_cruzada):.2f}"
)

Precisión del modelo: 0.73
Puntuaciones de validación cruzada: [0.71910112 0.80898876 0.69318182 0.64772727 0.71590909]
Promedio de puntuación de validación cruzada: 0.72
